# Segmentation test

In [1]:
import cv2
import numpy as np
import time
from IPython.display import clear_output

In [2]:
# Function to find angle between two vectors
def Angle(v1, v2):
    dot = np.dot(v1, v2)
    x_modulus = np.sqrt((v1 * v1).sum())
    y_modulus = np.sqrt((v2 * v2).sum())
    cos_angle = dot / x_modulus / y_modulus
    angle = np.degrees(np.arccos(cos_angle))
    return angle

# Function to find distance between two points in a list of lists
def FindDistance(A, B): 
    return np.sqrt(np.power((A[0][0] - B[0][0]), 2) + np.power((A[0][1] - B[0][1]), 2)) 

#Perform morphological transformations to filter out the background noise
def clean_background(frame):
    #Convert to HSV color space
    hsv_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    #Create a binary image with where white will be skin colors and rest is black
    mask = cv2.inRange(hsv_frame,np.array([2,50,50]),np.array([15,255,255]))
    
    #Kernel matrices for morphological transformation    
    kernel_square  = np.ones((11,11),np.uint8)
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    
    #Dilation increase skin color area
    #Erosion increase skin color area
    dilation  = cv2.dilate(mask,kernel_ellipse,iterations = 1)
    erosion   = cv2.erode(dilation,kernel_square,iterations = 1)    
    dilation2 = cv2.dilate(erosion,kernel_ellipse,iterations = 1)    
    filtered  = cv2.medianBlur(dilation2,5)
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8))
    dilation2 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilation3 = cv2.dilate(filtered,kernel_ellipse,iterations = 1)
    median    = cv2.medianBlur(dilation2,5)
    return cv2.threshold(median,127,255,0)[1]

In [7]:
#Open Camera object
cap = cv2.VideoCapture(0)

# Decrease frame size
# Note this may not work, only depends on camera support
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 200)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 200)
cap.set(cv2.CAP_PROP_FPS, 5)

# # Creating a window for HSV track bars
# cv2.namedWindow('HSV_TrackBar')

while(True):
    start_time = time.time()
    
    #Capture frame
    frame = cap.read()[1]

    threshold = clean_background(frame)
    
    #Find contours of the filtered frame
    _, contours, hierarchy = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)   
    
    #Draw Contours
#     cv2.drawContours(frame, contours, -1, (122,122,0), 3)
    
    #Find Max contour area (Assume that hand is in the frame)
    max_area = 250
#     ci = 0
 
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if(area > max_area):
            max_area = area
            ci = i  
    
    #Largest area contour  
    try:
        cnts = contours[ci]
    except IndexError:
        print("No contour found")
        break

    #Find convex hull
    hull = cv2.convexHull(cnts)
    
    #Find moments of the largest contour
    moments = cv2.moments(cnts)
    
    #Find convex defects
#     cv2.drawContours(frame, [hull], -1, (255, 255, 255), 2)
    
    #Central mass of first order moments
    if moments['m00'] != 0:
        cx = moments['m10'] // moments['m00'] # cx = M10/M00
        cy = moments['m01'] // moments['m00'] # cy = M01/M00
    centerMass=(cx, cy)    
    
    #Print bounding rectangle
    x, y, w, h = cv2.boundingRect(cnts)
    img = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    box = img[y:y+h, x:x+w]
    print(box.shape)
    box = cv2.resize(box, (200, 200), interpolation=cv2.INTER_CUBIC) 
    
    # Show final image
    cv2.imshow('Dilation', frame)
    cv2.imshow('Box', cv2.cvtColor(box, cv2.COLOR_BGR2GRAY))
    
    #Print execution time
    print("execution time per frame:\t{:.2f} [ms]".format((time.time()-start_time) * 1000))
    
    #close the output video by pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

(58, 60, 3)
execution time per frame:	629.21 [ms]
(209, 97, 3)
execution time per frame:	4.99 [ms]
(189, 109, 3)
execution time per frame:	12.34 [ms]
(188, 117, 3)
execution time per frame:	45.25 [ms]
(213, 115, 3)
execution time per frame:	32.18 [ms]
(190, 115, 3)
execution time per frame:	37.12 [ms]
(213, 112, 3)
execution time per frame:	20.94 [ms]
(96, 103, 3)
execution time per frame:	34.52 [ms]
(219, 117, 3)
execution time per frame:	19.48 [ms]
(212, 115, 3)
execution time per frame:	41.24 [ms]
(210, 114, 3)
execution time per frame:	26.50 [ms]
(182, 129, 3)
execution time per frame:	38.58 [ms]
(187, 127, 3)
execution time per frame:	20.68 [ms]
(214, 115, 3)
execution time per frame:	43.97 [ms]
(211, 117, 3)
execution time per frame:	28.72 [ms]
(214, 121, 3)
execution time per frame:	44.32 [ms]
(221, 120, 3)
execution time per frame:	29.27 [ms]
(214, 124, 3)
execution time per frame:	42.97 [ms]
(215, 110, 3)
execution time per frame:	29.90 [ms]
(217, 117, 3)
execution time per fr

(207, 138, 3)
execution time per frame:	39.08 [ms]
(207, 141, 3)
execution time per frame:	26.58 [ms]
(164, 119, 3)
execution time per frame:	42.98 [ms]
(206, 141, 3)
execution time per frame:	28.56 [ms]
(206, 139, 3)
execution time per frame:	44.76 [ms]
(208, 140, 3)
execution time per frame:	16.17 [ms]
(208, 137, 3)
execution time per frame:	30.46 [ms]
(206, 135, 3)
execution time per frame:	16.70 [ms]
(207, 139, 3)
execution time per frame:	34.87 [ms]
(206, 137, 3)
execution time per frame:	21.02 [ms]
(206, 142, 3)
execution time per frame:	35.09 [ms]
(204, 141, 3)
execution time per frame:	20.17 [ms]
(187, 135, 3)
execution time per frame:	36.80 [ms]
(205, 138, 3)
execution time per frame:	22.62 [ms]
(206, 140, 3)
execution time per frame:	38.36 [ms]
(224, 139, 3)
execution time per frame:	24.91 [ms]
(220, 137, 3)
execution time per frame:	40.78 [ms]
(207, 135, 3)
execution time per frame:	26.44 [ms]
(209, 139, 3)
execution time per frame:	42.10 [ms]
(208, 141, 3)
execution time pe

(179, 155, 3)
execution time per frame:	39.42 [ms]
(180, 154, 3)
execution time per frame:	23.50 [ms]
(181, 159, 3)
execution time per frame:	43.44 [ms]
(182, 147, 3)
execution time per frame:	28.12 [ms]
(178, 153, 3)
execution time per frame:	45.31 [ms]
(176, 146, 3)
execution time per frame:	30.33 [ms]
(178, 132, 3)
execution time per frame:	48.71 [ms]
(178, 156, 3)
execution time per frame:	17.76 [ms]
(160, 145, 3)
execution time per frame:	32.91 [ms]
(175, 131, 3)
execution time per frame:	18.28 [ms]
(172, 137, 3)
execution time per frame:	35.05 [ms]
(168, 133, 3)
execution time per frame:	21.47 [ms]
(167, 131, 3)
execution time per frame:	33.41 [ms]
(165, 122, 3)
execution time per frame:	19.12 [ms]
(157, 117, 3)
execution time per frame:	39.11 [ms]
(159, 125, 3)
execution time per frame:	22.74 [ms]
(172, 144, 3)
execution time per frame:	40.40 [ms]
(309, 178, 3)
execution time per frame:	26.82 [ms]
(208, 167, 3)
execution time per frame:	44.52 [ms]
(191, 130, 3)
execution time pe

(388, 285, 3)
execution time per frame:	34.96 [ms]
(381, 275, 3)
execution time per frame:	20.19 [ms]
(402, 280, 3)
execution time per frame:	40.07 [ms]
(387, 209, 3)
execution time per frame:	23.49 [ms]
(392, 290, 3)
execution time per frame:	41.99 [ms]
(370, 274, 3)
execution time per frame:	25.37 [ms]
(373, 239, 3)
execution time per frame:	43.62 [ms]
(381, 254, 3)
execution time per frame:	27.56 [ms]
(381, 253, 3)
execution time per frame:	44.96 [ms]
(362, 212, 3)
execution time per frame:	30.82 [ms]
(401, 283, 3)
execution time per frame:	31.75 [ms]
(375, 309, 3)
execution time per frame:	16.53 [ms]
(395, 293, 3)
execution time per frame:	33.24 [ms]
(383, 252, 3)
execution time per frame:	17.08 [ms]
(390, 194, 3)
execution time per frame:	35.54 [ms]
(373, 190, 3)
execution time per frame:	21.16 [ms]
(382, 195, 3)
execution time per frame:	38.46 [ms]
(331, 231, 3)
execution time per frame:	22.10 [ms]
(379, 201, 3)
execution time per frame:	35.07 [ms]
(338, 207, 3)
execution time pe

(351, 202, 3)
execution time per frame:	29.92 [ms]
(341, 185, 3)
execution time per frame:	30.54 [ms]
(361, 181, 3)
execution time per frame:	49.92 [ms]
(343, 183, 3)
execution time per frame:	18.14 [ms]
(344, 192, 3)
execution time per frame:	36.42 [ms]
(349, 194, 3)
execution time per frame:	20.91 [ms]
(358, 191, 3)
execution time per frame:	38.31 [ms]
(321, 200, 3)
execution time per frame:	22.98 [ms]
(325, 192, 3)
execution time per frame:	40.13 [ms]
(319, 188, 3)
execution time per frame:	26.17 [ms]
(322, 183, 3)
execution time per frame:	36.77 [ms]
(341, 201, 3)
execution time per frame:	20.91 [ms]
(326, 183, 3)
execution time per frame:	36.90 [ms]
(314, 187, 3)
execution time per frame:	21.94 [ms]
(317, 193, 3)
execution time per frame:	38.90 [ms]
(328, 190, 3)
execution time per frame:	26.93 [ms]
(318, 185, 3)
execution time per frame:	41.89 [ms]
(331, 180, 3)
execution time per frame:	26.89 [ms]
(335, 197, 3)
execution time per frame:	43.86 [ms]
(336, 191, 3)
execution time pe

-1

In [4]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1